# <font color=red>Лекция 6.4</font> <font color=blue>Иерархическая индексация</font>

   **Иерархическое индексирование** — это важная особенность pandas, поскольку она позволяет иметь несколько уровней индексов в одной оси. С ее помощью можно работать с данными в большом количестве измерений, по-прежнему используя для этого структуру данных из двух измерений (DataFrame). Это открывает двери для довольно сложного анализа данных и манипуляций, особенно для работы с более объемными данными.
   
   Иерархическая индексация (также называемая «многоуровневым» индексированием) была введена в выпуске pandas 0.4. *Узнать установленную версию pandas можно с помощью* `pd.__version__`.

In [ ]:
import pandas as pd
print (pd.__version__)

## <center>Еще раз об индексации в pandas</center>
Зная, что такое Series и Dataframes, и понимая как они устроены, проще разобраться со всеми их достоинствами. Главная особенность этих структур — наличие объекта Index, который в них интегрирован.

Объекты Index являются метками осей и содержат другие метаданные. Вы уже знаете, как массив с метками превращается в объект Index, и что для него нужно определить параметр index в конструкторе.

In [ ]:
ser = pd.Series([5,0,3,8,4], index=['red','blue','yellow','white','green'])
ser.index

В отличие от других элементов в структурах данных pandas (Series и Dataframe) объекты index — неизменяемые. Это обеспечивает безопасность, когда нужно передавать данные между разными структурами.

У каждого объекта Index есть методы и свойства, которые нужны, чтобы узнавать значения.

#### Методы Index

Есть методы для получения информации об индексах из структуры данных. Например, `idmin()` и `idmax()` — структуры, возвращающие индексы с самым маленьким и большим значениями.

In [ ]:
print(ser.idxmin())
print(ser.idxmax())

#### Индекс с повторяющимися метками
Пока что были только те случаи, когда у индексов одной структуры лишь одна, уникальная метка. Для большинства функций это обязательное условие, но не для структур данных pandas.

Определим, например, Series с повторяющимися метками.

In [ ]:
serd = pd.Series(range(6), index=['white','white','blue','green', 'green','yellow']) 
serd

Если метке соответствует несколько значений, то она вернет не один элемент, а объект Series.

In [ ]:
serd['white']

То же применимо и к Dataframe. При повторяющихся индексах он возвращает Dataframe.

В случае с маленькими структурами легко определять любые повторяющиеся индексы, но если структура большая, то растет и сложность этой операции. Для этого в pandas у объектов Index есть атрибут is_unique. Он сообщает, есть ли индексы с повторяющимися метками в структуре (Series или Dataframe).

In [ ]:
serd.index.is_unique


In [ ]:
frame = pd.DataFrame({'color' : ['blue', 'green', 'yellow', 'red', 'white'],
        'object' : ['ball', 'pen', 'pencil', 'paper', 'mug'],
        'price' : [1.2, 1.0, 0.6, 0.9, 1.7]})
frame.index.is_unique

## <center>Многоуровневая (иерархическая) индексация</center>
Начнем с простого примера, создав Series с двумя массивами индексов — структуру с двумя уровнями.

In [6]:
import pandas as pd
import numpy as np

mser = pd.Series(np.random.rand(8),
                 index=[['white','white','white','blue','blue','red','red','red'],
                        ['up','down','right','up','down','up','down','left']])
mser

white  up       0.133440
       down     0.581198
       right    0.596373
blue   up       0.261291
       down     0.503222
red    up       0.213611
       down     0.777228
       left     0.858796
dtype: float64

Проанализируем получившиеся индексы:

In [7]:
mser.index

MultiIndex([('white',    'up'),
            ('white',  'down'),
            ('white', 'right'),
            ( 'blue',    'up'),
            ( 'blue',  'down'),
            (  'red',    'up'),
            (  'red',  'down'),
            (  'red',  'left')],
           )

Как видно, имеем вложенную структуру структуру индексов.

За счет спецификации иерархического индексирования, выбор подмножеств значений в таком случае заметно упрощен. Можно выбрать значения для определенного значения первого индекса стандартным способом:

In [ ]:
mser['white']

Или же значения для конкретного значения во втором индекса — таким:

In [8]:
mser[:,'up']

white    0.133440
blue     0.261291
red      0.213611
dtype: float64

Если необходимо конкретное значение, просто указываются оба индекса.

In [ ]:
mser['white','up']

Иерархическое индексирование играет важную роль в изменении формы данных и групповых операциях, таких как сводные таблицы. Например, данные могут быть перестроены и использованы в объекте Dataframe с помощью функции `unstack()`. Она конвертирует Series с иерархическими индексами в простой Dataframe, где второй набор индексов превращается в новые колонки.

In [ ]:
mser.unstack()

Если необходимо выполнить обратную операцию — превратить Dataframe в Series, используется функция `stack()`.

Создадим новый DataFrame:

In [ ]:
frame = pd.DataFrame([[2, 1, 3, 9],[1, 6, 8, 5],[12, 9, 4, 11],[2, 4, 12, 5]],
                    columns=['Ball', 'Pen', 'Pensil','Paper'],
                    index=['red','blue','yellow','white'])


frame

И выполним над ним операцию `stack()`:

In [ ]:
frame.stack()

В Dataframe можно определить иерархическое индексирование для строк и колонок. Для этого необходимо определить массив массивов для параметров index и columns.

In [ ]:
mframe = pd.DataFrame(np.random.randn(16).reshape(4,4),
            index=[['white','white','red','red'], ['up','down','up','down']],
            columns=[['pen','pen','paper','paper'],[1,2,1,2]])
mframe

### Изменение порядка и сортировка уровней
Иногда потребуется поменять порядок уровней на оси или отсортировать значения на определенном уровне.

Функция `swaplevel()` принимает в качестве аргументов названия уровней, которые необходимо поменять относительно друг друга и возвращает новый объект с соответствующими изменениями, оставляя данные в том же состоянии.

In [ ]:
mframe.columns.names = ['objects','id']
mframe.index.names = ['colors','status']
mframe

In [ ]:
mframe.swaplevel('colors','status')

А функция `sort_index()` сортирует данные для конкретного уровня, указанного в параметрах.

In [ ]:
mframe.sort_index(level='colors')

### Общая статистика по уровню

У многих статистических методов для Dataframe есть параметр level, в котором нужно определить, для какого уровня нужно определить статистику.

Например, если нужна статистика для первого уровня, его нужно указать в параметрах.

In [ ]:
mframe.sum(level='colors')

Если же она необходима для конкретного уровня колонки, например, id, тогда требуется задать параметр axis и указать значение 1.

In [ ]:
mframe.sum(level='id', axis=1)